# Isprobavanje Faster R-CNN-a


In [ ]:
%%shell

pip install cython
# Install pycocotools, the version by default in Colab
# has a bug fixed in https://github.com/cocodataset/cocoapi/pull/354
pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-3dus4rd8
  Running command git clone -q https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-3dus4rd8
  Created wheel for pycocotools: filename=pycocotools-2.0-cp37-cp37m-linux_x86_64.whl size=265166 sha256=fbe019b0a157cc897bc2e83d748ff3176a0466283f9d87744cb15991add42ff8
  Stored in directory: /tmp/pip-ephem-wheel-cache-rvpq42bq/wheels/e2/6b/1d/344ac773c7495ea0b85eb228bc66daec7400a143a92d36b7b1
Successfully built pycocotools
  Attempting uninstall: pycocotools
    Found existing installation: pycocotools 2.0.4
    Uninstalling pycocotools-2.0.4:
      Successfully uninstalled pycocotools-2.0.4


In [ ]:
%%shell

# Download TorchVision repo to use some files from
# references/detection
git clone https://github.com/pytorch/vision.git
cd vision
git checkout v0.8.2

cp references/detection/utils.py ../
cp references/detection/transforms.py ../
cp references/detection/coco_eval.py ../
cp references/detection/engine.py ../
cp references/detection/coco_utils.py ../

Cloning into 'vision'...
remote: Enumerating objects: 148874, done.
remote: Counting objects: 100% (1250/1250), done.
remote: Compressing objects: 100% (238/238), done.
remote: Total 148874 (delta 1022), reused 1189 (delta 1007), pack-reused 147624
Receiving objects: 100% (148874/148874), 291.20 MiB | 30.54 MiB/s, done.
Resolving deltas: 100% (131612/131612), done.
Note: checking out 'v0.8.2'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at 2f40a483d [v0.8.X] .circleci: Add Python 3.9 to CI (#3063)


In [ ]:
from __future__ import print_function, division
from google.colab import output

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.io import read_image
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import random_split
from engine import train_one_epoch, evaluate
import utils
from PIL import Image, ImageDraw, ImageFont
import matplotlib.pyplot as plt
import time
import os
from os.path import exists
import copy
import transforms as T

from sklearn.metrics import confusion_matrix
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

import time
from time import sleep
from datetime import datetime
import random

plt.ion()   # interactive mode

In [ ]:
# Access to uploaded files
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
dir_prefix = 'drive/My Drive/Colab Notebooks/Diplomski'

In [ ]:
class Logger:
  def __init__(self, logger=None):
    self.logger = logger

  def log(self, msg):
    self._do(msg)
    if self.logger:
     self.logger.log(msg)

  def _do(self, msg):
    pass


class PrintLogger(Logger):
  def _do(self, msg):
    print(msg)

class TxtFileLogger(Logger):
  def __init__(self, path_to_file, logger=None):
    super().__init__(logger)
    self.path_to_file = path_to_file
    self.file = open(self.path_to_file, 'a')

  def _do(self, msg):
    self.file.write(msg + '\n')
    self.file.flush()

  def __del__(self):
    self.file.close()

In [ ]:
attrs_of_interest = ['bridge', 'endofilling', 'filling', 'crown']

def default_labeler(row):
  """ returns a list of labels which ranging from 1 to n, refering to a string label in array attrs_of_interest """
  label = []
  for i in range(len(attrs_of_interest)):
    if row[attrs_of_interest[i]] == 'yes': label.append(i + 1)
  return label

class XRayDataset(torch.utils.data.Dataset):
    def __init__(self, root, csv_file, subset=None, transforms=None, labeler=default_labeler):
        self.root = root
        self.transforms = transforms
        self.csv_file = csv_file
        self.subset = subset
        self.labeler = labeler
        print(f'Opening {csv_file}...')
        self.df = pd.read_csv(csv_file, sep = ";")
        if self.subset: self.df = self.subset(self.df)
        self.object_data_by_img = dict()

        for index, row in self.df.iterrows():       # Returns integer for a label if [] is returned the row is skipped
          label = self.labeler(row)
          for lbl in label:
            if not row['img_name'] in self.object_data_by_img:
              self.object_data_by_img[row['img_name']] = list()

            self.object_data_by_img[row['img_name']].append((lbl, row['x1'], row['y1'], row['x2'], row['y2']))

        # load all image files, sorting them to
        # ensure that they are aligned
        self.imgs = list(sorted(self.object_data_by_img))

    def __getitem__(self, idx):
        # load images and masks
        img_path = os.path.join(self.root, self.imgs[idx])
        img = Image.open(img_path).convert("RGB")

        # get bounding box coordinates for each object
        objs = self.object_data_by_img[self.imgs[idx]]
        num_objs = len(objs)
        labels = [x[0] for x in objs]
        boxes = [[x[1], x[2], x[3], x[4]] for x in objs] # Get x1, y1, x2, y2, bounding box coordinates of the tooth

        # convert everything into a torch.Tensor
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        # convert everything into a torch.Tensor
        labels = torch.as_tensor(labels, dtype=torch.int64)

        image_id = torch.tensor([idx])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        # suppose all instances are not crowd
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd

        if self.transforms is not None:
            img, target = self.transforms(img, target)

        return img, target

    def __len__(self):
        return len(self.imgs)

In [ ]:
# Func for more transforms if the need arises
def get_transform():
    transforms = []
    # converts the image, a PIL image, into a PyTorch Tensor
    transforms.append(T.ToTensor())
    return T.Compose(transforms)

In [ ]:
def keep_only_subset(df, num_of_total_negatives=-1, num_of_positives=-1):
  df_some_yes = df[~((df['crown'] == 'no') & (df['endofilling'] == 'no') & (df['filling'] == 'no') & (df['bridge'] == 'no'))]
  df_all_no = df[(df['crown'] == 'no') & (df['endofilling'] == 'no') & (df['filling'] == 'no') & (df['bridge'] == 'no')]
  if num_of_total_negatives >= 0:
    df_all_no = df_all_no.sample(n=num_of_total_negatives, random_state=1)
  if num_of_positives >= 0:
    df_some_yes = df_some_yes.sample(n=num_of_positives, random_state=1)
  print(f'len(df_some_yes): {len(df_some_yes)}, len(df_all_no): {len(df_all_no)}')
  return pd.concat([df_some_yes,df_all_no]).copy()

In [ ]:
img_transforms = transforms.Compose([transforms.ToTensor(), transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
image_datasets = dict()
dataloaders = dict()
dataset_sizes = dict()
class_names = dict()

csv_filename = os.path.join(dir_prefix, 'multi_label_dataset', 'corrected_tooth_info.csv')
img_dir = os.path.join(dir_prefix, 'data_store')
dataset = XRayDataset(root=img_dir,
                      csv_file=csv_filename,
                      subset=lambda df: keep_only_subset(df, num_of_total_negatives=0))


first = dataset[0]
print(first[1])
# total_number = len(dataset)
# split_size = [int(total_number * 0.7), total_number - int(total_number * 0.7)]
# dataset_train, dataset_val = random_split(dataset, split_size, generator=torch.Generator().manual_seed(42))

# image_datasets = {'train': dataset_train, 'val': dataset_val}
# dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
#                                              shuffle=True, num_workers=4)
#               for x in ['train', 'val']}
# dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
# class_names = attrs_of_interest

# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Opening drive/My Drive/Colab Notebooks/Diplomski/multi_label_dataset/corrected_tooth_info.csv...
len(df_some_yes): 9213, len(df_all_no): 0
{'boxes': tensor([[ 789.,  421.,  942.,  767.],
        [ 902.,  407., 1099.,  774.],
        [1486.,  404., 1609.,  811.],
        [2069.,  444., 2262.,  771.],
        [1539.,  801., 1629., 1137.],
        [1469.,  804., 1576., 1147.]]), 'labels': tensor([3, 3, 3, 3, 1, 4]), 'image_id': tensor([0]), 'area': tensor([52938., 72299., 50061., 63111., 30240., 36701.]), 'iscrowd': tensor([0, 0, 0, 0, 0, 0])}


In [ ]:
print(dataset.imgs[0])
print(len(dataset))

abaft-real-heady-goofy-gruesome-iron_22ff963593dc2d3e0310315f5bdf7cd8.jpg
932


In [ ]:
import math
import sys
import time

import torch
import torchvision.models.detection.mask_rcnn
import utils
from coco_eval import CocoEvaluator
from coco_utils import get_coco_api_from_dataset

@torch.inference_mode()
def evaluate_specific(model, data_loader, device, labels, iouThrs):
    n_threads = torch.get_num_threads()
    # FIXME remove this and make paste_masks_in_image run on the GPU
    torch.set_num_threads(1)
    cpu_device = torch.device("cpu")
    model.eval()
    metric_logger = utils.MetricLogger(delimiter="  ")
    header = "Test:"

    coco = get_coco_api_from_dataset(data_loader.dataset)
    iou_types = ["bbox"]
    coco_evaluator = CocoEvaluator(coco, iou_types)
    coco_evaluator.coco_eval["bbox"].params.catIds = labels # labels should be a list of classes ex. [1, 3] to analyze
    coco_evaluator.coco_eval["bbox"].params.iouThrs = iouThrs

    for images, targets in metric_logger.log_every(data_loader, 100, header):
        images = list(img.to(device) for img in images)

        if torch.cuda.is_available():
            torch.cuda.synchronize()
        model_time = time.time()
        outputs = model(images)

        outputs = [{k: v.to(cpu_device) for k, v in t.items()} for t in outputs]
        model_time = time.time() - model_time

        res = {target["image_id"].item(): output for target, output in zip(targets, outputs)}
        evaluator_time = time.time()
        coco_evaluator.update(res)
        evaluator_time = time.time() - evaluator_time
        metric_logger.update(model_time=model_time, evaluator_time=evaluator_time)

    # gather the stats from all processes
    metric_logger.synchronize_between_processes()
    print("Averaged stats:", metric_logger)
    coco_evaluator.synchronize_between_processes()

    # accumulate predictions from all images
    coco_evaluator.accumulate()
    coco_evaluator.summarize()
    torch.set_num_threads(n_threads)
    return coco_evaluator

In [ ]:
def get_5_cross_validation_splits(csv_file, nth_split=1):
  torch.manual_seed(1)
  dataset = XRayDataset(root=img_dir,
                      csv_file=csv_file,
                      subset=lambda df: keep_only_subset(df, num_of_total_negatives=0),
                      transforms=get_transform())
  dataset_test = XRayDataset(root=img_dir,
                      csv_file=csv_file,
                      subset=lambda df: keep_only_subset(df, num_of_total_negatives=0),
                      transforms=get_transform())

  step=186
  start = 186*(nth_split - 1)
  end = start + step
  if nth_split == 5: end += 2

  indices = torch.randperm(len(dataset)).tolist()
  dataset = torch.utils.data.Subset(dataset, [x for i,x in enumerate(indices) if i not in range(start,end)])
  dataset_test = torch.utils.data.Subset(dataset_test, indices[start:end])
  return dataset, dataset_test

def get_train_test_datasets(csv_file, len_train):
  torch.manual_seed(1)
  dataset = XRayDataset(root=img_dir,
                      csv_file=csv_file,
                      subset=lambda df: keep_only_subset(df, num_of_total_negatives=0),
                      transforms=get_transform())
  dataset_test = XRayDataset(root=img_dir,
                      csv_file=csv_file,
                      subset=lambda df: keep_only_subset(df, num_of_total_negatives=0),
                      transforms=get_transform())

  # split the dataset in train and test set
  indices = torch.randperm(len(dataset)).tolist()
  dataset = torch.utils.data.Subset(dataset, indices[:-len_train])
  dataset_test = torch.utils.data.Subset(dataset_test, indices[-len_train:])
  return dataset, dataset_test


In [ ]:
def main(model, dataset_csv_file, nth_split):
    # train on the GPU or on the CPU, if a GPU is not available
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

    # use our dataset and defined transformations
    dataset, dataset_test = get_5_cross_validation_splits(csv_file=dataset_csv_file, nth_split=nth_split)

    # define training and validation data loaders
    data_loader = torch.utils.data.DataLoader(
        dataset, batch_size=2, shuffle=True, num_workers=4,
        collate_fn=utils.collate_fn)

    data_loader_test = torch.utils.data.DataLoader(
        dataset_test, batch_size=1, shuffle=False, num_workers=4,
        collate_fn=utils.collate_fn)

    # move model to the right device
    model.to(device)

    # construct an optimizer
    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = torch.optim.SGD(params, lr=0.005,
                                momentum=0.9, weight_decay=0.0005)
    # and a learning rate scheduler
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                                   step_size=3,
                                                   gamma=0.1)

    # let's train it for 10 epochs
    num_epochs = 10

    for epoch in range(num_epochs):
        # train for one epoch, printing every 10 iterations
        train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
        # update the learning rate
        lr_scheduler.step()
    # evaluate on the test dataset
    evaluate(model, data_loader_test, device=device)

    print("That's it!")

## Training model with corrected dataset

In [ ]:
print(f"Training model on corrected dataset")
# load a model pre-trained on COCO
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

# replace the classifier with a new one, that has
# num_classes which is user-defined
num_classes = 5  # 0 background class + 4 tooth interventions
# get number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features
# replace the pre-trained head with a new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)


corrected_data_csv = os.path.join(dir_prefix, 'multi_label_dataset', 'corrected_tooth_info.csv')
main(model, dataset_csv_file=corrected_data_csv, nth_split=5)

# datetime object containing current date and time
now = datetime.now()

# dd/mm/YY H:M:S
dt_string = now.strftime("%d:%m:%Y-%H:%M:%S")

torch.save(model, os.path.join(dir_prefix, 'trained_models', f'corrected_data_faster-rcnn_model-split{5}-{dt_string}-epochs-10'))

Training model on corrected dataset
len(df_some_yes): 9213, len(df_all_no): 0
len(df_some_yes): 9213, len(df_all_no): 0


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch: [0]  [  0/372]  eta: 0:15:49  lr: 0.000018  loss: 3.9818 (3.9818)  loss_classifier: 1.8271 (1.8271)  loss_box_reg: 0.2461 (0.2461)  loss_objectness: 1.8247 (1.8247)  loss_rpn_box_reg: 0.0840 (0.0840)  time: 2.5526  data: 1.8116  max mem: 3056
Epoch: [0]  [ 10/372]  eta: 0:04:45  lr: 0.000153  loss: 2.6634 (2.7281)  loss_classifier: 1.6829 (1.5883)  loss_box_reg: 0.3649 (0.4233)  loss_objectness: 0.3370 (0.6637)  loss_rpn_box_reg: 0.0513 (0.0528)  time: 0.7890  data: 0.1957  max mem: 3320
Epoch: [0]  [ 20/372]  eta: 0:04:05  lr: 0.000288  loss: 1.9148 (2.1780)  loss_classifier: 1.0286 (1.1870)  loss_box_reg: 0.5293 (0.5196)  loss_objectness: 0.1926 (0.4264)  loss_rpn_box_reg: 0.0400 (0.0450)  time: 0.6041  data: 0.0315  max mem: 3320
Epoch: [0]  [ 30/372]  eta: 0:03:47  lr: 0.000422  loss: 1.4861 (1.9426)  loss_classifier: 0.6550 (1.0132)  loss_box_reg: 0.6003 (0.5573)  loss_objectness: 0.1432 (0.3279)  loss_rpn_box_reg: 0.0373 (0.0442)  time: 0.5958  data: 0.0281  max mem: 3320


In [ ]:
for i in [5]:
  print(f"Training model on nth_split={i}")
  # load a model pre-trained on COCO
  model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

  # replace the classifier with a new one, that has
  # num_classes which is user-defined
  num_classes = 5  # 0 background class + 4 tooth interventions
  # get number of input features for the classifier
  in_features = model.roi_heads.box_predictor.cls_score.in_features
  # replace the pre-trained head with a new one
  model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

  nth_split = i
  main(model, nth_split)

  # datetime object containing current date and time
  now = datetime.now()

  # dd/mm/YY H:M:S
  dt_string = now.strftime("%d:%m:%Y-%H:%M:%S")

  torch.save(model, os.path.join(dir_prefix, 'trained_models', f'faster-rcnn_model-split{nth_split}-{dt_string}-epochs-10'))

Training model on nth_split=5


Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth


  0%|          | 0.00/160M [00:00<?, ?B/s]

len(df_some_yes): 8790, len(df_all_no): 0
len(df_some_yes): 8790, len(df_all_no): 0


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch: [0]  [  0/372]  eta: 0:29:53  lr: 0.000018  loss: 3.5929 (3.5929)  loss_classifier: 1.8270 (1.8270)  loss_box_reg: 0.1983 (0.1983)  loss_objectness: 1.4940 (1.4940)  loss_rpn_box_reg: 0.0736 (0.0736)  time: 4.8204  data: 3.0763  max mem: 2210
Epoch: [0]  [ 10/372]  eta: 0:09:48  lr: 0.000153  loss: 2.8614 (2.7655)  loss_classifier: 1.7096 (1.6637)  loss_box_reg: 0.2787 (0.4007)  loss_objectness: 0.4095 (0.6500)  loss_rpn_box_reg: 0.0492 (0.0511)  time: 1.6270  data: 0.3082  max mem: 2472
Epoch: [0]  [ 20/372]  eta: 0:08:37  lr: 0.000288  loss: 2.0351 (2.2188)  loss_classifier: 1.1924 (1.2713)  loss_box_reg: 0.5040 (0.5056)  loss_objectness: 0.1670 (0.4006)  loss_rpn_box_reg: 0.0367 (0.0413)  time: 1.3024  data: 0.0306  max mem: 2472
Epoch: [0]  [ 30/372]  eta: 0:08:11  lr: 0.000422  loss: 1.5075 (1.9773)  loss_classifier: 0.7034 (1.0744)  loss_box_reg: 0.6163 (0.5636)  loss_objectness: 0.0917 (0.2998)  loss_rpn_box_reg: 0.0290 (0.0395)  time: 1.3305  data: 0.0448  max mem: 2472


In [ ]:
print('Evaluating model trained on corrected dataset.')
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Load the model
# corrected_data_faster-rcnn_model-split5-24:05:2022-14:33:30-epochs-10
# faster-rcnn_model-split5-08:05:2022-18:34:51-epochs-10
model_ld = torch.load(os.path.join(dir_prefix, 'trained_models', 'corrected_data_faster-rcnn_model-split5-24:05:2022-14:33:30-epochs-10'))
model_ld.eval()

# use our corrected dataset
corrected_data_csv = os.path.join(dir_prefix, 'multi_label_dataset', 'corrected_tooth_info.csv')
normal_data_csv = os.path.join(dir_prefix, 'multi_label_dataset', 'tooth_info.csv')
_, dataset_test = get_5_cross_validation_splits(csv_file=corrected_data_csv, nth_split=5)

data_loader_test = torch.utils.data.DataLoader(
    dataset_test, batch_size=1, shuffle=False, num_workers=4,
    collate_fn=utils.collate_fn)

for i in [1,2,3,4]:
  print(f'Model evaluation for {attrs_of_interest[i-1]}:')
  evaluate_specific(model_ld, data_loader_test, device, labels=[i], iouThrs=[.3])
  print()

Evaluating model trained on corrected dataset.
Opening drive/My Drive/Colab Notebooks/Diplomski/multi_label_dataset/corrected_tooth_info.csv...
len(df_some_yes): 9213, len(df_all_no): 0
Opening drive/My Drive/Colab Notebooks/Diplomski/multi_label_dataset/corrected_tooth_info.csv...
len(df_some_yes): 9213, len(df_all_no): 0


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Model evaluation for bridge:
creating index...
index created!
Test:  [  0/188]  eta: 0:03:27  model_time: 0.1972 (0.1972)  evaluator_time: 0.0145 (0.0145)  time: 1.1056  data: 0.8456  max mem: 673
Test:  [100/188]  eta: 0:00:16  model_time: 0.1089 (0.1164)  evaluator_time: 0.0014 (0.0018)  time: 0.1762  data: 0.0277  max mem: 673
Test:  [187/188]  eta: 0:00:00  model_time: 0.1087 (0.1157)  evaluator_time: 0.0011 (0.0018)  time: 0.1580  data: 0.0201  max mem: 673
Test: Total time: 0:00:34 (0.1816 s / it)
Averaged stats: model_time: 0.1087 (0.1157)  evaluator_time: 0.0011 (0.0018)
Accumulating evaluation results...
DONE (t=0.01s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.30:0.30 | area=   all | maxDets=100 ] = 0.827
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.30:0.30 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP

In [ ]:
font = ImageFont.truetype("LiberationMono-Bold.ttf", size=60)
font_small = ImageFont.truetype("LiberationMono-Bold.ttf", size=30)
font2 = ImageFont.truetype("LiberationMono-Bold.ttf", size=80)

In [ ]:
# train on the GPU or on the CPU, if a GPU is not available
model_ld = torch.load(os.path.join(dir_prefix, 'trained_models', 'faster-rcnn_model-split5-08:05:2022-18:34:51-epochs-10'))
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

normal_data_csv = os.path.join(dir_prefix, 'multi_label_dataset', 'tooth_info.csv')
_, dataset_test = get_5_cross_validation_splits(csv_file=normal_data_csv, nth_split=5)
n = 5
# pick n images from the test set
indexes = random.sample(range(len(dataset_test)), n)

for i in indexes:
  img, t = dataset_test[i]
  # put the model in evaluation mode
  model_ld.eval()
  with torch.no_grad():
      prediction = model_ld([img.to(device)])

  prediction = prediction[0]
  print("Prediction:")
  print(prediction)
  print("Target:")
  print(t)

  img_pred = Image.fromarray(img.mul(255).permute(1, 2, 0).byte().numpy())
  imgd = ImageDraw.Draw(img_pred)
  for i, s in enumerate(prediction['scores']):
    if s < 0.51: continue
    imgd.rectangle(prediction['boxes'][i].tolist(), outline ="blue", width=4)
    imgd.text(prediction['boxes'][i].tolist()[:2], str(int(prediction['labels'][i])), font=font, align ="left")
  imgd.text((20,20), 'Prediction', font=font2, align ="left", fill='blue')
  display(img_pred)

  img_target = Image.fromarray(img.mul(255).permute(1, 2, 0).byte().numpy())
  imgdt = ImageDraw.Draw(img_target)
  font = ImageFont.truetype("LiberationMono-Bold.ttf", size=60)
  already_seen = dict()
  for i, l in enumerate(t['labels']):
    imgdt.rectangle(t['boxes'][i].tolist(), outline ="blue", width=4)
    txt_coords = tuple(t['boxes'][i].tolist()[:2])
    shift = already_seen.get(txt_coords, 0) * 45
    final_coords = (txt_coords[0] + shift, txt_coords[1])
    imgdt.text(final_coords, str(int(l)), font=font, align ="left")
    already_seen[txt_coords] = already_seen.get(txt_coords, 0) + 1
  imgdt.text((20,20), 'Target', font=font2, align ="left", fill='blue')
  display(img_target)


In [ ]:
def approve_data_corrections(model, dataset, columns, score_thres, nth_split, begin=0, display_size=(1271, 600), logger=PrintLogger(),):
  """"function fot the Data correction approval process")
  After each case you will be asked whether to approve or reject the data correction
  1 => approve, 0 => reject, exit => terminate the approval process"""

  now = datetime.now()
  # dd/mm/YY H:M:S
  dt_string = now.strftime("%d:%m:%Y")
  logger.log(f"Starting at: {dt_string}")
  logger.log("Welcome to the data correction approval process!")
  logger.log("After each case you will be asked whether to approve or reject the data correction")
  logger.log("1 => approve, 0 => reject, exit, or any other string => terminate the approval process\n")

  underlying_data_path = os.path.join(dir_prefix, 'multi_label_dataset', 'tooth_info.csv') # Used for extracting info about teeth without any recorded interventions (those aren't present in dataset.dataset)
  underlying_data = pd.read_csv(underlying_data_path, sep = ";")

  df = pd.DataFrame(columns=columns)
  csv_path = os.path.join(dir_prefix, 'corrections_csvs', f'THRES-{score_thres}-split={nth_split}.csv')
  if exists(csv_path):
    df = pd.read_csv(csv_path, sep=';')

  try:
    for iter in range(begin, len(dataset)):
      img, t = dataset_test[iter]

      img_name = dataset.dataset.imgs[int(t['image_id'])]
      logger.log(f'\niter={iter}\nExamining {img_name}...')

      if (df['img_name'] == img_name).any():
        logger.log(f'Skipping image since it is already present in corrections...\n')
        continue

      # put the model in evaluation mode
      model_ld.eval()
      with torch.no_grad():
          prediction = model_ld([img.to(device)])

      prediction = prediction[0]
      b = prediction['boxes'].tolist()
      lbl = prediction['labels'].tolist()
      s = prediction['scores'].tolist()
      above_thres = [(lbl[i], b[i]) for i, s in enumerate(s) if s >= score_thres]
      if len(above_thres) == 0: continue

      b_t = t['boxes'].tolist()
      lbl_t = t['labels'].tolist()
      targets = [(l, b_t[i]) for i, l in enumerate(lbl_t)]

      suspicious_not_in_targets = []
      for attr_l in [1,2,3,4]:
        curr_targets = list(filter(lambda x: x[0] == attr_l, targets))
        curr_above_thres = list(filter(lambda x: x[0] == attr_l, above_thres))

        if len(curr_above_thres) == 0: continue
        if len(curr_targets) == 0:
          suspicious_not_in_targets.extend(curr_above_thres)
          continue

        t_boxes = torch.tensor([x[1] for x in curr_targets])
        pred_boxes = torch.tensor([x[1] for x in curr_above_thres])

        ious_matrix = torchvision.ops.box_iou(pred_boxes, t_boxes)
        for i, ious in enumerate(ious_matrix):
          if not (ious > 0.2).any(): suspicious_not_in_targets.append(curr_above_thres[i])


      if len(suspicious_not_in_targets) == 0:
        logger.log('No suspicious omissions detected.')
        logger.log('Moving onto next image...\n')
        continue

      img_pred = Image.fromarray(img.mul(255).permute(1, 2, 0).byte().numpy())
      imgd = ImageDraw.Draw(img_pred)
      for i,case in enumerate(suspicious_not_in_targets):
        imgd.rectangle(case[1], outline ="red", width=4)
        text_coords = case[1][:2]
        imgd.text(text_coords, str(int(case[0])), font=font, align ="left", fill='red')
        imgd.text([text_coords[0], text_coords[1] - 30], f'ith={i+1}', font=font_small, align ="left", fill='red')
      imgd.text((20,20), 'High confidence predictions not in Target', font=font2, align ="left", fill='blue')

      img_pred = img_pred.resize(display_size, Image.ANTIALIAS)
      display(img_pred)
      sleep(0.5)

      decision = input('decision(1 / 0 / S*): ')
      if decision == '1' or decision[0] == 'S':               # APPROVE

        if decision == '1': decision = '1' * len(suspicious_not_in_targets)
        if decision[0] == 'S': decision = decision[1:]
        if not len(decision) == len(suspicious_not_in_targets):
          logger.log('Inputed binary string of different length than suspicious_not_in_targets.')
          logger.log('Correction proposal rejected.\n')
        else:
          all_teeth_curr_img = underlying_data[underlying_data['img_name'] == img_name]
          curr_teeth_bb = []
          for index, row in all_teeth_curr_img.iterrows():
            curr_teeth_bb.append([row['x1'], row['y1'], row['x2'], row['y2']])
          curr_teeth_bb = torch.tensor(curr_teeth_bb)
          sus_bb = torch.tensor([x[1] for x in suspicious_not_in_targets])

          ious_matrix = torchvision.ops.box_iou(sus_bb, curr_teeth_bb)
          for i, ious in enumerate(ious_matrix):
            curr_attr = attrs_of_interest[suspicious_not_in_targets[i][0] - 1]
            corresponding_tooth_idx = int(torch.argmax(ious))
            corresponding_tooth_row = all_teeth_curr_img.iloc[corresponding_tooth_idx].copy()
            tooth_id = corresponding_tooth_row['tooth_id']

            if decision[i] == '0':
              logger.log(f"For {tooth_id} {curr_attr} correction was rejected. Moving on...")
              continue

            if not corresponding_tooth_row[curr_attr] == 'yes':
              logger.log(f"For {tooth_id} {curr_attr} was set to {corresponding_tooth_row[curr_attr]}, setting it to 'yes'...")
              corresponding_tooth_row[curr_attr] = 'yes'
              df = df.append(corresponding_tooth_row, ignore_index=True)
            else:
              logger.log(f"For {tooth_id} {curr_attr} was already set to 'yes'. Moving on...")
          logger.log('Correction recorded.\n')

      elif decision == '0':             # REJECT
        logger.log('Correction proposal rejected.\n')
      else:                             # END APPROVAL PROCESS AND SAVE PROGRESS
        logger.log('Exiting...\n')
        break
      # output.clear()
  except Exception as e:
    print('An Exception occured:')
    print(e)
    print('Saving finalised corrections...')

  df.to_csv(csv_path, sep=';', index=False)
  logger.log('Approved corrections updated.\n')



In [ ]:
nth_split=5

# train on the GPU or on the CPU, if a GPU is not available
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model_ld = torch.load(os.path.join(dir_prefix, 'trained_models', 'faster-rcnn_model-split5-08:05:2022-18:34:51-epochs-10'))
model_ld.eval()


score_thres = 0.9
path_to_log = os.path.join(dir_prefix, 'logs', 'correction_approval', f'corrections-score_thres={score_thres}-split={nth_split}.txt')
log = PrintLogger(TxtFileLogger(path_to_log))
log.log(f'Corrections from split={nth_split}')

_, dataset_test = get_5_cross_validation_splits(nth_split=nth_split)
approve_data_corrections(model_ld, dataset_test, columns=dataset_test.dataset.df.columns, score_thres=score_thres, nth_split=nth_split, logger=log, begin=60)
del log

In [ ]:
import matplotlib
system_fonts = matplotlib.font_manager.findSystemFonts(fontpaths=None, fontext='ttf')

In [ ]:
system_fonts

['/usr/share/fonts/truetype/liberation/LiberationSerif-Italic.ttf',
 '/usr/share/fonts/truetype/liberation/LiberationSansNarrow-Italic.ttf',
 '/usr/share/fonts/truetype/liberation/LiberationSerif-Regular.ttf',
 '/usr/share/fonts/truetype/liberation/LiberationMono-Bold.ttf',
 '/usr/share/fonts/truetype/liberation/LiberationMono-BoldItalic.ttf',
 '/usr/share/fonts/truetype/liberation/LiberationSans-Regular.ttf',
 '/usr/share/fonts/truetype/liberation/LiberationSerif-Bold.ttf',
 '/usr/share/fonts/truetype/liberation/LiberationSerif-BoldItalic.ttf',
 '/usr/share/fonts/truetype/liberation/LiberationSansNarrow-BoldItalic.ttf',
 '/usr/share/fonts/truetype/liberation/LiberationSansNarrow-Regular.ttf',
 '/usr/share/fonts/truetype/liberation/LiberationMono-Regular.ttf',
 '/usr/share/fonts/truetype/liberation/LiberationSansNarrow-Bold.ttf',
 '/usr/share/fonts/truetype/liberation/LiberationSans-Italic.ttf',
 '/usr/share/fonts/truetype/humor-sans/Humor-Sans.ttf',
 '/usr/share/fonts/truetype/liberat